In [1]:
# 整个思路 模型脱敏（结构化）通过categorical——embedder方式——神经网络，自行搭建测试——k折验证，自主调参——(可以加正则化优化)
# 导入数据
import pandas as pd
import numpy as np
import categorical_embedder as ce
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
import matplotlib.pyplot as plt
import csv

Using TensorFlow backend.


In [2]:
train_data = pd.read_csv('train_estimates.csv')
train_data.shape

(35000, 36)

In [3]:
train_data.head()

,carid,tradeTime,brand,serial,model,mileage,color,cityId,carCode,transferCount,...,anonymousFeature7,anonymousFeature8,anonymousFeature9,anonymousFeature10,anonymousFeature11,anonymousFeature12,anonymousFeature13,anonymousFeature14,anonymousFeature15,price
0,1,2021/6/28,1,1,1,4.01,1,1,1,0,...,0,1,5,2,1,4220*1740*1625,201709,1,0,4.24
1,2,2021/6/25,2,2,2,8.60,1,2,1,0,...,0,2,4,3,1+2,4630*1775*1480,201609,2,0,7.38
2,5,2021/6/19,5,5,5,15.56,1,2,3,0,...,0,0,0,0,0,4515*1725*1445,0,2,0,1.00
3,6,2021/6/29,6,6,6,6.04,1,3,1,3,...,2018/8/18,2,5,2,1,4500*1834*1707,201608,2,0,4.38
4,7,2021/6/30,7,7,7,5.70,4,1,2,2,...,2020/9/20,1,5,2,1,4315*1783*1606,201204,1,0,5.90


In [4]:
# 数据脱敏(encoding)
X = train_data.drop(['carid', 'price'], axis=1)
y = train_data['carid']

In [5]:
# 确定分类变量
embedding_info = ce.get_embedding_info(X)
embedding_info

{'tradeTime': (682, 50),
 'registerDate': (204, 50),
 'licenseDate': (3836, 50),
 'anonymousFeature7': (2140, 50),
 'anonymousFeature11': (7, 4),
 'anonymousFeature12': (2293, 50),
 'anonymousFeature15': (2219, 50)}

In [6]:
# 整数编码
X_encoded, encoders = ce.get_label_encoded_data(X)
X_encoded.head()

,tradeTime,brand,serial,model,mileage,color,cityId,carCode,transferCount,seatings,...,anonymousFeature6,anonymousFeature7,anonymousFeature8,anonymousFeature9,anonymousFeature10,anonymousFeature11,anonymousFeature12,anonymousFeature13,anonymousFeature14,anonymousFeature15
0,580,1,1,1,4.01,1,1,1,0,5,...,1,0,1,5,2,1,237,201709,1,0
1,577,2,2,2,8.60,1,2,1,0,5,...,2,0,2,4,3,2,1108,201609,2,0
2,570,5,5,5,15.56,1,2,3,0,5,...,2,0,0,0,0,0,791,0,2,0
3,581,6,6,6,6.04,1,3,1,3,5,...,1,1137,2,5,2,1,749,201608,2,0
4,583,7,7,7,5.70,4,1,2,2,5,...,1,1812,1,5,2,1,373,201204,1,0


In [7]:
# 分割数据
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y)

# ce.get_embeddings trains NN, extracts embeddings and return a dictionary containing the embeddings
embeddings = ce.get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info, is_classification=True,
                               epochs=100, batch_size=256)

Training:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch 0:   0%|          | 0/21000 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/21000 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/21000 [00:00<?, ?it/s]

Epoch 3:   0%|          | 0/21000 [00:00<?, ?it/s]

...

Epoch 95:   0%|          | 0/21000 [00:00<?, ?it/s]

Epoch 96:   0%|          | 0/21000 [00:00<?, ?it/s]

Epoch 97:   0%|          | 0/21000 [00:00<?, ?it/s]

Epoch 98:   0%|          | 0/21000 [00:00<?, ?it/s]

Epoch 99:   0%|          | 0/21000 [00:00<?, ?it/s]

In [9]:
embeddings

{'tradeTime': array([[-2.7953753,  2.8196225, -2.8119745, ...,  2.819222 ,  2.795289 ,
          2.7749248],
        [-4.317521 ,  4.2907257, -4.3178368, ...,  4.314686 ,  4.290268 ,
          4.261467 ],
        [-4.2639294,  4.2675376, -4.205001 , ...,  4.2600117,  4.3015914,
          4.287869 ],
        ...,
        [-6.2244225,  6.274782 , -6.2770433, ...,  6.226404 ,  6.31681  ,
          6.276836 ],
        [-5.2048655,  5.137553 , -5.1570883, ...,  5.2160397,  5.2303777,
          5.1994643],
        [-5.4082875,  5.4080396, -5.4242797, ...,  5.4359775,  5.471535 ,
          5.4262176]], dtype=float32),
 'registerDate': array([[ 1.3858063,  1.3475266, -1.3975576, ...,  1.3160911,  1.3811802,
          1.4118932],
        [ 1.4012289,  1.3999584, -1.4110327, ...,  1.3889124,  1.3211637,
          1.3760346],
        [ 1.3142023,  1.3362013, -1.3991772, ...,  1.3210043,  1.3485087,
          1.4045192],
        ...,
        [ 2.4911885,  2.468034 , -2.5074015, ...,  2.4440322,  2

In [10]:
train_data_encoded = ce.get_embeddings_in_dataframe(embeddings = embeddings,encoders = encoders)

  0%|          | 0/7 [00:00<?, ?it/s]

In [11]:
train_data_encoded['tradeTime']

,tradeTime_embedding_0,tradeTime_embedding_1,tradeTime_embedding_2,tradeTime_embedding_3,tradeTime_embedding_4,tradeTime_embedding_5,tradeTime_embedding_6,tradeTime_embedding_7,tradeTime_embedding_8,tradeTime_embedding_9,...,tradeTime_embedding_40,tradeTime_embedding_41,tradeTime_embedding_42,tradeTime_embedding_43,tradeTime_embedding_44,tradeTime_embedding_45,tradeTime_embedding_46,tradeTime_embedding_47,tradeTime_embedding_48,tradeTime_embedding_49
2020/1/1,-2.795375,2.819623,-2.811975,-2.808209,2.800533,2.823267,-2.846039,2.849105,2.818124,-2.805276,...,-2.854001,2.770102,-2.820168,2.839991,-2.797694,-2.829575,2.797089,2.819222,2.795289,2.774925
2020/1/10,-4.317521,4.290726,-4.317837,-4.322490,4.291134,4.294250,-4.249851,4.328335,4.271351,-4.253001,...,-4.250378,4.290586,-4.255593,4.250992,-4.239377,-4.240359,4.324435,4.314686,4.290268,4.261467
2020/1/11,-4.263929,4.267538,-4.205001,-4.300253,4.289303,4.235409,-4.204054,4.232375,4.228931,-4.235854,...,-4.221120,4.288479,-4.275506,4.275586,-4.286389,-4.259864,4.207788,4.260012,4.301591,4.287869
2020/1/12,-4.365170,4.410783,-4.381144,-4.376383,4.403898,4.356318,-4.361753,4.354600,4.392757,-4.438703,...,-4.422056,4.374206,-4.403617,4.344091,-4.374018,-4.355794,4.347727,4.347612,4.397804,4.387970
2020/1/13,-3.310848,3.254492,-3.269339,-3.261480,3.258914,3.239386,-3.295339,3.276741,3.287254,-3.252850,...,-3.281953,3.229008,-3.329284,3.275736,-3.269723,-3.256707,3.295443,3.297295,3.273645,3.291085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021/9/5,-5.716943,5.755322,-5.688557,-5.728567,5.760036,5.784210,-5.745849,5.786541,5.735513,-5.778637,...,-5.790246,5.702575,-5.751111,5.770620,-5.777793,-5.694810,5.741557,5.785418,5.767978,5.710990
2021/9/6,-6.412296,6.449572,-6.386394,-6.377770,6.417446,6.363235,-6.354682,6.357053,6.382372,-6.405718,...,-6.425656,6.401253,-6.445770,6.391811,-6.397356,-6.412768,6.425796,6.401430,6.358559,6.415920
2021/9/7,-6.224422,6.274782,-6.277043,-6.239089,6.267501,6.321920,-6.271574,6.317708,6.241437,-6.278219,...,-6.290853,6.277377,-6.239893,6.266316,-6.276702,-6.242121,6.252997,6.226404,6.316810,6.276836
2021/9/8,-5.204865,5.137553,-5.157088,-5.169534,5.131296,5.218737,-5.164563,5.174657,5.192889,-5.201501,...,-5.196457,5.224280,-5.221486,5.208110,-5.158416,-5.154804,5.167676,5.216040,5.230378,5.199464


In [12]:
train_data_encoded['anonymousFeature7']

,anonymousFeature7_embedding_0,anonymousFeature7_embedding_1,anonymousFeature7_embedding_2,anonymousFeature7_embedding_3,anonymousFeature7_embedding_4,anonymousFeature7_embedding_5,anonymousFeature7_embedding_6,anonymousFeature7_embedding_7,anonymousFeature7_embedding_8,anonymousFeature7_embedding_9,...,anonymousFeature7_embedding_40,anonymousFeature7_embedding_41,anonymousFeature7_embedding_42,anonymousFeature7_embedding_43,anonymousFeature7_embedding_44,anonymousFeature7_embedding_45,anonymousFeature7_embedding_46,anonymousFeature7_embedding_47,anonymousFeature7_embedding_48,anonymousFeature7_embedding_49
0,-13.451273,13.465025,13.455828,13.442896,-13.436273,-13.446111,-13.478346,13.474504,-13.449364,-13.466134,...,-13.434599,13.468612,13.466931,-13.459965,-13.468732,13.431492,13.442351,13.470064,-13.443416,-13.451605
2008/12/15,-0.047517,-0.023585,0.046383,-0.048520,-0.027186,-0.000568,0.010626,0.018945,0.008280,0.001082,...,-0.044817,0.038981,0.001365,-0.032302,0.019829,0.031504,-0.018989,0.034961,0.025033,0.026776
2008/2/15,-1.321515,1.311306,1.315640,1.311637,-1.322050,-1.396410,-1.398129,1.362280,-1.375931,-1.315818,...,-1.325314,1.359838,1.350613,-1.335046,-1.367118,1.377032,1.362180,1.313615,-1.316689,-1.363250
2008/2/2,-1.313600,1.307098,1.306237,1.401973,-1.325252,-1.355621,-1.330481,1.312136,-1.398798,-1.333145,...,-1.341161,1.319690,1.323496,-1.356365,-1.374196,1.355259,1.356245,1.373778,-1.369919,-1.401294
2008/3/31,-1.394838,1.319199,1.361448,1.352151,-1.405593,-1.392917,-1.333967,1.397441,-1.347596,-1.400889,...,-1.355487,1.345655,1.334983,-1.401927,-1.410618,1.397019,1.362679,1.396047,-1.397395,-1.389656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022/12/24,-0.010601,-0.001564,0.024218,0.032708,0.006307,-0.009946,0.007343,0.042387,0.048303,-0.022386,...,-0.005267,0.010153,0.032871,0.020234,0.020938,-0.009016,0.010756,0.038920,-0.039659,-0.021325
2022/2/28,-0.028200,0.017554,-0.014963,0.043011,-0.027790,0.006947,-0.026516,0.031851,0.017574,0.013463,...,0.002405,0.022117,-0.036384,-0.027236,0.013635,-0.032845,-0.013217,0.027612,0.004300,0.020243
2022/3/29,-1.357135,1.399037,1.317385,1.371114,-1.352656,-1.349481,-1.411032,1.393846,-1.382957,-1.391417,...,-1.349192,1.408443,1.395711,-1.313363,-1.333241,1.399858,1.365075,1.313659,-1.409237,-1.408254
2022/4/13,-1.288550,1.252034,1.255524,1.252993,-1.307307,-1.278602,-1.311216,1.308174,-1.303341,-1.309583,...,-1.239517,1.251659,1.305423,-1.247561,-1.229510,1.271070,1.260462,1.315978,-1.242540,-1.294145


In [13]:
train_data_regular = ce.fit_transform(X, embeddings=embeddings, encoders=encoders, drop_categorical_vars=True)
train_data_regular.head()

  0%|          | 0/7 [00:00<?, ?it/s]

,brand,serial,model,mileage,color,cityId,carCode,transferCount,seatings,country,...,anonymousFeature15_embedding_40,anonymousFeature15_embedding_41,anonymousFeature15_embedding_42,anonymousFeature15_embedding_43,anonymousFeature15_embedding_44,anonymousFeature15_embedding_45,anonymousFeature15_embedding_46,anonymousFeature15_embedding_47,anonymousFeature15_embedding_48,anonymousFeature15_embedding_49
0,1,1,1,4.01,1,1,1,0,5,779413,...,-13.471803,-13.487217,-13.46157,-13.496195,-13.497024,13.47478,13.465812,-13.463632,13.473866,13.490201
1,2,2,2,8.60,1,2,1,0,5,779415,...,-13.471803,-13.487217,-13.46157,-13.496195,-13.497024,13.47478,13.465812,-13.463632,13.473866,13.490201
2,5,5,5,15.56,1,2,3,0,5,0,...,-13.471803,-13.487217,-13.46157,-13.496195,-13.497024,13.47478,13.465812,-13.463632,13.473866,13.490201
3,6,6,6,6.04,1,3,1,3,5,779413,...,-13.471803,-13.487217,-13.46157,-13.496195,-13.497024,13.47478,13.465812,-13.463632,13.473866,13.490201
4,7,7,7,5.70,4,1,2,2,5,779415,...,-13.471803,-13.487217,-13.46157,-13.496195,-13.497024,13.47478,13.465812,-13.463632,13.473866,13.490201


In [14]:
train_data_regular.to_csv('result.csv')
